In [ ]:
!python -m pip install --upgrade setuptools
!pip install git+https://github.com/tky823/ssspy.git

In [ ]:
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import IPython.display as ipd

In [ ]:
from ssspy.utils.dataset import download_sample_speech_data

In [ ]:
n_sources = 2
sample_rate = 16000
max_samples = 10 * sample_rate
sisec2010_tag = "dev1_female3"
n_fft, hop_length = 4096, 2048

In [ ]:
waveform_src_img = download_sample_speech_data(
    n_sources=n_sources,
    sisec2010_tag=sisec2010_tag,
    max_samples=max_samples,
    conv=True,
) # (n_channels, n_sources, n_samples)
waveform_mix = np.sum(waveform_src_img, axis=1)  # (n_channels, n_samples)

In [ ]:
for idx, waveform in enumerate(waveform_mix):
    print("Mixture: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
from ssspy.transform import whiten
from ssspy.algorithm import projection_back
from ssspy.bss.fdica import NaturalGradLaplaceFDICA

## Holonomic type

In [ ]:
fdica = NaturalGradLaplaceFDICA(
    step_size=1e-1,
    is_holonomic=True,
)
print(fdica)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_est = fdica(spectrogram_mix, n_iter=500)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(fdica.loss)
plt.show()
plt.close()

## Non-holonomic type

In [ ]:
fdica = NaturalGradLaplaceFDICA(
    step_size=1e-1,
    is_holonomic=False,
    scale_restoration=False
)
print(fdica)

In [ ]:
_, _, spectrogram_mix = ss.stft(waveform_mix, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
spectrogram_mix_whitened = whiten(spectrogram_mix)
spectrogram_est = fdica(spectrogram_mix_whitened, n_iter=500)
spectrogram_est = projection_back(spectrogram_est, reference=spectrogram_mix)

In [ ]:
_, waveform_est = ss.istft(spectrogram_est, window="hann", nperseg=n_fft, noverlap=n_fft-hop_length)

In [ ]:
for idx, waveform in enumerate(waveform_est):
    print("Estimated source: {}".format(idx + 1))
    display(ipd.Audio(waveform, rate=sample_rate))
    print()

In [ ]:
plt.figure()
plt.plot(fdica.loss)
plt.show()
plt.close()